<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Import pandas for data handling and plotting

# --- 1. Environmental & Global Parameters ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=100):
        self.total_sim_steps = total_sim_steps
        self.days_per_cycle = days_per_cycle # Steps per "day" cycle
        self.time = 0 # Current time step (index)

        # UV Profile (simplified sine wave for circadian cycle)
        # np.linspace creates an array with 'total_sim_steps' points, so indices 0 to total_sim_steps-1
        self.uv_intensity = np.sin(np.linspace(0, 2 * np.pi * (total_sim_steps / days_per_cycle), total_sim_steps))
        self.uv_intensity[self.uv_intensity < 0] = 0 # No negative UV, night is 0

        # --- External Resource Pools (These MUST be present here) ---
        self.external_ntp_pool = 10000.0
        self.external_aa_pool = 10000.0
        self.external_trp_tyr_cys_pool = 500.0 # Scarce, as per hypothesis

    def get_current_uv(self):
        # Ensure we don't go out of bounds at the very end of simulation
        if self.time >= self.total_sim_steps:
            return 0.0 # Return 0 if simulation time somehow exceeds array bounds
        return self.uv_intensity[self.time]

    def step(self):
        self.time += 1
        # Corrected: Use self.days_per_cycle as defined in __init__
        self.current_day_step = self.time % self.days_per_cycle

# --- 2. Proto-Cell/System Class ---
class ProtoCell:
    def __init__(self, env):
        self.env = env

        # Core Molecular Pools (as concentrations/amounts)
        self.rna_pool = 100.0 # General RNA (templates, proto-rRNA)
        self.dna_pool = 10.0  # Basic DNA (more stable template)
        self.atp_pool = 50.0  # Energy currency
        self.mcyw_peptide_pool = 5.0 # Foraging/Antenna peptides
        self.other_peptide_pool = 2.0 # Non-mcyw peptides

        # Resource Stocks (internal to the proto-cell)
        self.internal_ntp_stock = 100.0
        self.internal_aa_stock = 100.0
        self.internal_trp_tyr_cys_stock = 3.0 # DECREASED for more scarcity (was 5.0)

        # System States & Damage
        self.rna_damage_level = 0.0 # Accumulates from UV
        self.protein_synthesis_active = True # Whether ribosomes are active
        self.rna_dna_copying_active = True # Whether replication is active
        self.uv_safety_status = "SAFE" # Based on internal signals
        self.premature_stop_signal = 0.0 # Initialize here, updated in apply_uv_damage

        # --- Parameters (tuned to GUARANTEE cyclical "ARRESTED" state) ---
        # UV Damage parameters
        self.uv_damage_rate_per_uv = 0.25 # SIGNIFICANTLY INCREASE: Make UV much more damaging (was 0.15)
        self.damage_tolerance_threshold = 3.0 # DECREASE: Make the threshold for arrest much lower (was 4.0)

        # Foraging parameters
        self.foraging_efficiency_mcyw = 0.15 # Keep as is (from previous tuning)
        self.foraging_atp_cost = 0.005 # Keep as is (from previous tuning)
        self.trp_tyr_cys_foraging_bias = 8.0 # Keep as is (from previous tuning)

        # Protein Synthesis parameters
        self.protein_synthesis_rate = 0.12 # Keep as is (from previous tuning)
        self.protein_synthesis_ntp_cost = 0.1
        self.protein_synthesis_aa_cost = 0.1
        self.protein_synthesis_atp_cost = 0.02
        self.mcyw_synthesis_bias = 0.5 # Keep high to ensure mcyw production (from previous tuning)

        # Stop Codon Precariousness/Signaling
        self.precarious_codon_rate_per_damage = 0.3 # INCREASE: Make damage-induced stops even more potent (was 0.2)
        self.premature_stop_signal_threshold = 0.35 # DECREASE: Make the trigger for arrest more sensitive (was 0.4)
                                                     # This is the overall 'danger_metric' threshold for arrest

        # ATP Synthesis
        self.atp_generation_rate_per_mcyw_uv = 0.05 # Keep as is (from previous tuning)

        # Repair parameters
        self.repair_rate = 0.1 # Keep as is (from previous tuning)
        self.repair_atp_cost = 0.01
        self.trp_tyr_cys_for_repair_needed = 0.1 # Keep as is

        # RNA/DNA Copying parameters
        self.rna_dna_copy_rate = 0.03 # Keep as is (from previous tuning)
        self.rna_dna_copy_ntp_cost = 0.05
        self.rna_dna_copy_atp_cost = 0.01

        # Degradation rates
        self.rna_degradation_rate = 0.0008 # Keep as is (from previous tuning)
        self.peptide_degradation_rate = 0.0015 # Keep as is (from previous tuning)
        self.atp_degradation_rate = 0.003 # Keep as is (from previous tuning)

    # --- Core Processes ---
    def _apply_uv_damage(self):
        current_uv = self.env.get_current_uv()
        if current_uv > 0:
            damage_increase = current_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            # Simulate the "safety reportage" signal from damaged precarious codons
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else:
            self.premature_stop_signal = 0 # No new stops at night

    def _forage(self):
        if self.mcyw_peptide_pool <= 0 or self.atp_pool <= self.foraging_atp_cost:
            return # Cannot forage without mcyw or ATP

        # Consume ATP for foraging movement
        atp_consumed = self.foraging_atp_cost * self.mcyw_peptide_pool
        self.atp_pool -= atp_consumed
        self.atp_pool = max(0, self.atp_pool) # Ensure not negative

        # Acquire general AAs and NTPs from the Environment's external pools
        general_foraged = self.foraging_efficiency_mcyw * self.mcyw_peptide_pool
        transfer_ntp = min(general_foraged / 2, self.env.external_ntp_pool) # Assume half NTP, half AA
        transfer_aa = min(general_foraged / 2, self.env.external_aa_pool)
        self.internal_ntp_stock += transfer_ntp
        self.env.external_ntp_pool -= transfer_ntp
        self.internal_aa_stock += transfer_aa
        self.env.external_aa_pool -= transfer_aa

        # Prioritize rare Trp/Tyr/Cys from the Environment's external pool
        rare_foraged = general_foraged * self.trp_tyr_cys_foraging_bias # Higher efficiency for these
        transfer_rare = min(rare_foraged, self.env.external_trp_tyr_cys_pool)
        self.internal_trp_tyr_cys_stock += transfer_rare
        self.env.external_trp_tyr_cys_pool -= transfer_rare


    def _synthesize_protein(self):
        if not self.protein_synthesis_active:
            return

        required_ntp_base = self.protein_synthesis_ntp_cost
        required_aa_base = self.protein_synthesis_aa
